In [152]:
import pandas as pd
import numpy as np
import re
import math
import xlrd
import datetime

In [138]:
#test_csv = pandas.read_csv('Folder 3 - 1_18.csv')
test_xls = pandas.ExcelFile('/Users/mikey/Downloads/6down.xlsx')

position = '6'

In [ ]:
tab1 = pd.read_excel(test_xls, 0)
tab1.head()

In [ ]:
def convertDate(date):
    date = '2019-' + date[0:-2] + '-' + date[-2:]
    return date
#date = '2019-' + test_date[-4:].replace('/', '-')

In [ ]:
# Turn a hh:mm:ss string to number of seconds
# hh: piece is optional

HHMMSS_PAT = re.compile("(\d+)??:??(\d+):(\d+)$")
def hhmmss_to_seconds(s):
    m = HHMMSS_PAT.match(s)
    g = m.groups()
    s = 0
    n = len(g)
    for i in range(n):
        if g[n-i-1] is not None:
            s += 60**i * int(g[n-i-1])
    return s

In [ ]:
def convertTime(time):
    if time == time:
        converted = []
        time = time.replace(" ", "")
        for item in time.split(','):
            time_to_convert = item.split('/')[0]
            converted.append(hhmmss_to_seconds(time_to_convert))
    else:
        converted = []
    return converted

In [188]:
mouse_labels = ('mouse1', 'mouse2', 'mouse3', 'mouse4', 'mouse5', 'mouse6')

In [287]:
def parseSheet(test_csv, date, position, mouse_labels):
    test_dict = {}
    log = []
    
    #Check that date location is correct
    try:
        isinstance(test_csv.columns.values[1], datetime.datetime)
    except:
            return {}, ['Date (or file more generally) is possibly not formatted correctly']
    
    #Check that Video information is in expected location in sheet
    try:
        isinstance(test_csv.loc[2][0], str)
        #print(position, date, test_csv.loc[2][0])
    except:
            return {}, ['File is possibly not formatted correctly']
    
    #Extract annotation info and convert into usable format
    for item in test_csv.iterrows():
        if item[0]>3 and item[1][0] is np.nan:       #An empty row indicates end of confident annotations
            break
        elif item[0]>3:
            try:          
                seizures = {}
                date_dict = {}
                pos_dict = {}
                for i, key in enumerate(mouse_labels):
                    seizures[key] = convertTime(item[1][i+2])
                date_dict[date] = seizures
                pos_dict[position] = date_dict
                test_dict[item[1][0]] = pos_dict
            except Exception as error:
                    log_key = item[1][0] + '_' + 'line' + str(item[0] + 2) + '  ' + str(error)
                    log.append(log_key)
                    print('There was an issue with the annotation for sheet %s, movie %s' % (date, item[1][0]))
                
    return test_dict, log

In [288]:
#Generate position_dict

params_dict = {
    
    '3': (('66-2', '70-3', '72-2'), '/Users/mikey/Downloads/Folder3.xlsx'),
    '4': (('65-3', '74-2','71-2', '65-1'), '/Users/mikey/Downloads/Folder4.xlsx'),
    '6 up': (('74-1', '68-2', '65-2', '73-2', '70-1', '73-3'), '/Users/mikey/Downloads/6UP.xlsx'),
    '6 down': (('73-1', '75-2', '66-1', '69-3', '70-2', '71-1'), '/Users/mikey/Downloads/6down.xlsx')
    
}

In [291]:
#Extract annotations for all videos
all_sheets = {}
error_log = {'3':{}, '4':{}, '6 up':{}, '6 down':{}}

for position, value in params_dict.items():
    filename = value[1]
    mouse_labels = value[0]
    
    test_xls = pandas.ExcelFile(filename)  
    
    for sheet in test_xls.sheet_names:
        this_sheet = pandas.read_excel(test_xls, sheet_name=sheet)
        date = convertDate(sheet)
        #position = '3'
        new_dict, log = parseSheet(this_sheet, date, position, mouse_labels)
        if new_dict:
            all_sheets.update(new_dict)
        if log:
            error_log[position].update({date:log})
    

There was an issue with the annotation for sheet 2019--28, movie M2U00408
There was an issue with the annotation for sheet 2019--29, movie M2U00433
There was an issue with the annotation for sheet 2019-2-10, movie M2U00440
There was an issue with the annotation for sheet 2019-1-18, movie M2U00041
There was an issue with the annotation for sheet 2019-1-20, movie M2U00015
There was an issue with the annotation for sheet 2019-1-22, movie M2U00043
There was an issue with the annotation for sheet 2019-1-28, movie M2U00172
There was an issue with the annotation for sheet 2019-1-31, movie M2U00231
There was an issue with the annotation for sheet 2019--25, movie M2U00346
There was an issue with the annotation for sheet 2019--26, movie M2U00364
There was an issue with the annotation for sheet 2019-2-10, movie M2U00440
There was an issue with the annotation for sheet 2019-2-11, movie M2U00467
There was an issue with the annotation for sheet 2019-2-19, movie M2U000625
There was an issue with the 

In [280]:
new_dict

{'MAH00079': {'6 down': {'2019-219-19': {'73-1': [],
    '75-2': [632],
    '66-1': [470],
    '69-3': [],
    '70-2': [3457],
    '71-1': []}}},
 'MAH00080': {'6 down': {'2019-219-19': {'73-1': [],
    '75-2': [1433, 4466],
    '66-1': [],
    '69-3': [],
    '70-2': [],
    '71-1': []}}},
 'MAH00081': {'6 down': {'2019-219-19': {'73-1': [],
    '75-2': [1407, 2267, 2880],
    '66-1': [],
    '69-3': [],
    '70-2': [1268, 3513],
    '71-1': []}}}}

In [292]:
error_log

{'3': {'2019--28': ["M2U00408_line24  'NoneType' object has no attribute 'groups'"],
  '2019--29': ["M2U00433_line27  'NoneType' object has no attribute 'groups'"],
  '2019-2-10': ["M2U00440_line9  'NoneType' object has no attribute 'groups'"]},
 '4': {'2019-1-18': ["M2U00041_line14  'NoneType' object has no attribute 'groups'"],
  '2019-1-20': ["M2U00015_line19  'NoneType' object has no attribute 'groups'"],
  '2019-1-22': ["M2U00043_line6  'NoneType' object has no attribute 'groups'"],
  '2019-1-28': ["M2U00172_line13  'NoneType' object has no attribute 'groups'"],
  '2019-1-31': ["M2U00231_line12  'NoneType' object has no attribute 'groups'"],
  '2019--25': ["M2U00346_line22  'NoneType' object has no attribute 'groups'"],
  '2019--26': ["M2U00364_line20  'NoneType' object has no attribute 'groups'"],
  '2019-2-10': ["M2U00440_line9  'NoneType' object has no attribute 'groups'"],
  '2019-2-11': ["M2U00467_line13  'NoneType' object has no attribute 'groups'"],
  '2019-2-19': ["M2U0006

In [204]:
#Export json 
import json
with open('seizure_annotations_test.json', 'w') as outfile:
    json.dump(all_sheets, outfile)

In [ ]:
test_dict = parseSheet(test_csv, )